# ai_classify()

In [0]:
%sql
-- URGENT vs NON-URGENT Demo (Perfect for ai_classify() - Binary Choice)
CREATE OR REPLACE TABLE lesteve_sandbox.bronze.urgency_tickets (
  ticket_id INT, 
  customer_name STRING,
  description STRING,
  created_date TIMESTAMP
) USING DELTA;

-- 16 tickets: 8 Urgent + 8 Non-Urgent + edge cases
INSERT INTO lesteve_sandbox.bronze.urgency_tickets VALUES 
-- URGENT (8 tickets - Production down, revenue loss, safety)
(1, 'Acme Factory', 'Production line STOPPED - no liftgate, 500k/hr lost, need IMMEDIATE truck', '2026-01-08 14:30:00'),
(2, 'PharmaCorp', 'VACCINES at 45F - batch RUINED, FDA audit tomorrow', '2026-01-08 15:15:00'),
(3, 'AutoPlant', 'Assembly line DOWN - missing pallet jack, 200 cars delayed', '2026-01-08 16:00:00'),
(4, 'Hospital', 'Critical MEDS stuck at dock - patient care at risk', '2026-01-08 16:45:00'),
(5, 'DataCenter', 'Server rack delivery BLOCKED - liftgate failed, cloud outage imminent', '2026-01-08 17:20:00'),
(6, 'OilRig', 'HAZMAT chemicals delayed - safety violation, shutdown imminent', '2026-01-08 17:55:00'),
(7, 'RetailHQ', 'Black Friday inventory WRONG - 1M revenue impact today', '2026-01-08 18:30:00'),
(8, 'Airline', 'Aircraft parts SHORT - 3 flights cancelled, FAA fines', '2026-01-08 19:05:00'),

-- NON-URGENT (8 tickets - Billing, preferences, documentation)
(9, 'OfficeSupplies', 'Wrong paper color delivered, reorder next week', '2026-01-08 09:00:00'),
(10, 'Marketing', 'Brochures arrived early, prefer next batch Wednesday', '2026-01-08 10:30:00'),
(11, 'HR', 'Employee handbook update - missing page 47', '2026-01-08 11:15:00'),
(12, 'Legal', 'Contract copy missing signature page, scan tomorrow', '2026-01-08 12:00:00'),
(13, 'IT', 'Extra cables arrived, return when convenient', '2026-01-08 13:45:00'),
(14, 'Facilities', 'Office chairs slightly different model, exchange next month', '2026-01-08 14:00:00'),
(15, 'Sales', 'Promo swag pens low ink, reorder standard time', '2026-01-08 14:15:00'),
(16, 'Finance', 'Invoice typo in PO number, fix on next billing cycle', '2026-01-08 14:20:00');

-- PERFECT ai_classify() use case: Binary urgent/not urgent
SELECT 
  ticket_id, customer_name, description,
  ai_classify(description, ARRAY('Urgent','Not Urgent')) AS urgency_level
FROM lesteve_sandbox.bronze.urgency_tickets
ORDER BY ticket_id;


ticket_id,customer_name,description,urgency_level
1,Acme Factory,"Production line STOPPED - no liftgate, 500k/hr lost, need IMMEDIATE truck",Urgent
2,PharmaCorp,"VACCINES at 45F - batch RUINED, FDA audit tomorrow",Urgent
3,AutoPlant,"Assembly line DOWN - missing pallet jack, 200 cars delayed",Urgent
4,Hospital,Critical MEDS stuck at dock - patient care at risk,Urgent
5,DataCenter,"Server rack delivery BLOCKED - liftgate failed, cloud outage imminent",Urgent
6,OilRig,"HAZMAT chemicals delayed - safety violation, shutdown imminent",Urgent
7,RetailHQ,Black Friday inventory WRONG - 1M revenue impact today,Urgent
8,Airline,"Aircraft parts SHORT - 3 flights cancelled, FAA fines",Urgent
9,OfficeSupplies,"Wrong paper color delivered, reorder next week",Not Urgent
10,Marketing,"Brochures arrived early, prefer next batch Wednesday",Not Urgent


# ai_query()

### Accessorial Categorization

In [0]:
%sql
-- Accessorials ai_query() Demo 
CREATE OR REPLACE TABLE lesteve_sandbox.bronze.accessorials (
  id INT, attribute STRING
) USING DELTA;

INSERT INTO lesteve_sandbox.bronze.accessorials VALUES 
-- EQUIPMENT (Liftgate + Pallet Jack) - 9 variants
(1, "Liftgate Service - Handling - CarrierA - Midwest - NoExp - NoTemp"),
(2, "lift gate service - Handling - CarrierB - Midwest - NoExp - NoTemp"),
(3, "Lift-Gate Service Charge - Handling - CarrierC - Midwest - NoExp - NoTemp"),
(4, "Service Liftgate Handling - CarrierA - Midwest - NoExp - NoTemp"),
(5, "Handling CarrierB Lift Gate Service - Midwest - NoExp - NoTemp"),
(6, "LIFtgate - Handling - CarrierA - MidWest - NoExp - NoTemp"),
(15, "Pallet Jack Usage - Handling - Warehouse1 - Midwest - NoExp - NoTemp"),
(16, "Pallet Jack Use Fee - Handling - Warehouse1 - Midwest - NoExp - NoTemp"),
(17, "Usage Pallet Jack Fee - Warehouse1 - Midwest - NoExp - NoTemp"),

-- RESIDENTIAL DELIVERY - 6 variants  
(8, "Residential Delivery - Delivery - CarrierA - Northeast - NoExp - NoTemp"),
(9, "res. delivery charge - Delivery - CarrierC - Northeast - NoExp - NoTemp"),
(10, "Delivery Fee Residential - CarrierB - Northeast - NoExp - NoTemp"),
(11, "Residential Delivery Fee - Delivery - CarrierB - Northeast - NoExp - NoTemp"),
(27, "Residential Delvry Fee - Delivery - CarrierA - Northeast - NoExp - NoTemp"),
(28, "Residential Delivery Extra Charge - Delivery - CarrierA - Northeast - NoExp - NoTemp"),

-- EXPEDITED - 5 variants
(12, "Expedited Shipping - Delivery - CarrierA - South - Exp - NoTemp"),
(13, "Ship Fee expedited - Delivery - CarrierB - South - Exp - NoTemp"),
(14, "Exp Ship Fee - Delivery - CarrierA - South - Exp - NoTemp"),
(29, "Express Shipping Service - Delivery - CarrierB - South - Exp - NoTemp"),
(30, "Express Ship Fee - Delivery - CarrierC - South - Exp - NoTemp"),

-- TEMPERATURE CONTROL - 4 variants
(18, "Temperature Controlled Fee - Handling - CarrierC - West - NoExp - Temp"),
(19, "Temp Ctrl Fee - Handling - CarrierC - West - NoExp - Temp"),
(20, "Temp Controlled Handling Fee - CarrierC - West - NoExp - Temp"),
(31, "Temperature Control Charge - Handling - CarrierC - West - NoExp - Temp"),

-- BILLING ADJUSTMENT - 3 variants
(21, "Freight Bill Correction - Adjustment - BillingSys - All - NoExp - NoTemp"),
(22, "Freight Bill Corr. Fee - Adjustment - BillingSys - All - NoExp - NoTemp"),
(23, "Bill Correction Freight - Adjustment - BillingSys - All - NoExp - NoTemp"),

-- SNACK - 4 variants
(32, "Seasonal Salad - Appetizer - OutletA - Downtown - NoExp - NoTemp"),
(33, "Salad Mix with Nuts - Appetizer - OutletB - Uptown - NoExp - NoTemp"),
(34, "Fresh Green Salad with Dressings - Appetizer - OutletA - Downtown - NoExp - NoTemp"),
(35, "Classic Tomato Pie - Main - OutletC - Midtown - NoExp - NoTemp"),

-- BEVERAGE - 5 variants
(46, "Lemon Drink - Beverage - OutletA - Downtown - NoExp - NoTemp"),
(47, "Lemon Citrus Juice - Beverage - OutletA - Downtown - NoExp - NoTemp"),
(48, "Fresh Lemon Drink - Beverage - OutletA - Downtown - NoExp - NoTemp"),
(49, "Iced Herbal Tea - Beverage - OutletA - Downtown - NoExp - NoTemp"),
(50, "Herbal Tea Iced - Beverage - OutletA - Downtown - NoExp - NoTemp"),

-- EDGE CASES (Manual Review Required) - 5 variants
(99, "Quantum Widget Unloading Service - FutureTech - West - NoExp - NoTemp"),
(100, "AI Robot Arm Fee - Automation - RobotCo - All - NoExp - NoTemp"),
(101, "Blockchain Verification Surcharge - Tech - CryptoSys - All - NoExp - NoTemp"),
(102, "NFT Artwork Crating Fee - SpecialHandling - GalleryX - NoExp - NoTemp"),
(103, "Celebrity Autograph Authentication - Premium - VIPService - NoExp - NoTemp");

-- Production ai_query() Classification (Clean output, no ** formatting)
SELECT 
  id, attribute,
  REGEXP_REPLACE(
    TRIM(
      ai_query(
        'databricks-claude-opus-4-5',
    CONCAT(
          'Classify this 3PL accessorial service into EXACTLY ONE category only: ',
          '"Equipment", "Residential Delivery", "Expedited", "Temperature Control", "Billing Adjustment", "Food/Beverage/Snack". ',
          'For non-standard services ',
          'respond ONLY "Manual Review Required". ',
          'Return ONLY the category name. No bold, no asterisks, no markdown, no explanation. ',
          'Accessorial: ', attribute
        )
      )
    ),
    '[\\*\\_\\n\\r\\s]+', ' '
  ) AS billable_category
FROM lesteve_sandbox.bronze.accessorials
ORDER BY id;


id,attribute,billable_category
1,Liftgate Service - Handling - CarrierA - Midwest - NoExp - NoTemp,Equipment
2,lift gate service - Handling - CarrierB - Midwest - NoExp - NoTemp,Equipment
3,Lift-Gate Service Charge - Handling - CarrierC - Midwest - NoExp - NoTemp,Equipment
4,Service Liftgate Handling - CarrierA - Midwest - NoExp - NoTemp,Equipment
5,Handling CarrierB Lift Gate Service - Midwest - NoExp - NoTemp,Equipment
6,LIFtgate - Handling - CarrierA - MidWest - NoExp - NoTemp,Equipment
8,Residential Delivery - Delivery - CarrierA - Northeast - NoExp - NoTemp,Residential Delivery
9,res. delivery charge - Delivery - CarrierC - Northeast - NoExp - NoTemp,Residential Delivery
10,Delivery Fee Residential - CarrierB - Northeast - NoExp - NoTemp,Residential Delivery
11,Residential Delivery Fee - Delivery - CarrierB - Northeast - NoExp - NoTemp,Residential Delivery


### HR Resume Screening by Skill Match

In [0]:
%sql
-- HR Resume Screening by Skill Match 
CREATE OR REPLACE TABLE lesteve_sandbox.bronze.job_applications (
  candidate_name STRING, experience_summary STRING
) USING DELTA;

-- 20 resumes: Every skill 4x + 4 edge cases
INSERT INTO lesteve_sandbox.bronze.job_applications VALUES 
-- SPARK EXPERT (4 candidates)
('John Doe', '5yr Spark SQL Databricks, built lakehouse pipelines, PySpark ETL jobs'),
('Sarah Lee', 'Spark Developer with 3 years experience, optimized Delta Lake performance'),
('Mike Chen', 'Senior Spark Engineer, distributed ML pipelines on Databricks clusters'),
('Lisa Patel', 'Spark Streaming specialist, real-time data processing pipelines'),

-- SQL POWER USER (4 candidates)
('Tom Wilson', '10yr SQL expert, complex analytics queries, Power BI semantic models'),
('Anna Ruiz', 'SQL database administrator, performance tuning, indexing strategies'),
('Raj Singh', 'Advanced SQL analytics, window functions, CTEs, data warehousing'),
('Emily Park', 'SQL for business intelligence, dashboard optimization, query refactoring'),

-- DATABRICKS PRO (4 candidates)
('David Kim', 'Databricks Certified, Unity Catalog implementation, Lakehouse architecture'),
('Rachel Green', 'Databricks migration specialist, legacy Hadoop to Lakeflow pipelines'),
('Carlos Lopez', 'Databricks governance expert, attribute-based access control'),
('Megan Wong', 'Databricks AI Functions, ai_query production pipelines, Agent Bricks'),

-- FABRIC SPECIALIST (4 candidates)
('Brian Taylor', 'Microsoft Fabric expert, semantic models, dataflows, lakehouse'),
('Kelly Adams', 'Power BI Fabric integration, real-time analytics, shortcuts'),
('James Brown', 'Fabric OneLake governance, cross-workspace data sharing'),
('Nina Gupta', 'Fabric capacity management, cost optimization, workload orchestration'),

-- EDGE CASES / UNKNOWNS (4 candidates → Manual Review)
('Quantum Zhang', 'Quantum computing algorithms, qubit optimization frameworks'),  -- Unrelated tech
('Art Historian', 'Renaissance painting analysis, art provenance research'),       -- Wrong industry
('Crypto Trader', 'DeFi smart contract auditing, blockchain MEV strategies'),     -- Finance
('Music Producer', 'Audio engineering, Ableton Live mastering, sound design');     -- Creative
-- Replace your ai_classify with this single ai_query (no custom logic needed)
SELECT 
  candidate_name, experience_summary,
  REGEXP_REPLACE(
    ai_query(
      'databricks-claude-opus-4-5',
      CONCAT(
        'Classify this resume into EXACTLY ONE category: Spark Expert, SQL Power User, Databricks Pro, Fabric Specialist. ',
        'If no clear match, respond ONLY "Manual Review Required".\n\n',
        'Resume: ', experience_summary
      )
    ),
    '\\*', ''
  ) AS primary_skill
FROM lesteve_sandbox.bronze.job_applications
ORDER BY candidate_name;

candidate_name,experience_summary,primary_skill
Anna Ruiz,"SQL database administrator, performance tuning, indexing strategies",SQL Power User
Art Historian,"Renaissance painting analysis, art provenance research",Manual Review Required
Brian Taylor,"Microsoft Fabric expert, semantic models, dataflows, lakehouse",Fabric Specialist
Carlos Lopez,"Databricks governance expert, attribute-based access control",Databricks Pro
Crypto Trader,"DeFi smart contract auditing, blockchain MEV strategies",Manual Review Required
David Kim,"Databricks Certified, Unity Catalog implementation, Lakehouse architecture",Databricks Pro
Emily Park,"SQL for business intelligence, dashboard optimization, query refactoring",SQL Power User
James Brown,"Fabric OneLake governance, cross-workspace data sharing",Fabric Specialist
John Doe,"5yr Spark SQL Databricks, built lakehouse pipelines, PySpark ETL jobs",Databricks Pro
Kelly Adams,"Power BI Fabric integration, real-time analytics, shortcuts",Fabric Specialist


# ai_extract()

In [0]:
%sql
-- ai_extract() Demo: Extract from MESSY 3PL Tickets - Real-World Chaos
CREATE OR REPLACE TABLE lesteve_sandbox.bronze.ai_extract_tickets (
  ticket_id INT, 
  customer_name STRING,
  description STRING,
  created_date TIMESTAMP
) USING DELTA;

-- 12 MESSY tickets with chaotic embedded entities (no clean patterns)
INSERT INTO lesteve_sandbox.bronze.ai_extract_tickets VALUES 
(1, 'AcmeCorp', 'stuck CNTR-ABC-123 at LB port!!! need LG LTL \$12k 12KUSD liftgate midwest', '2026-01-08 14:30:00'),
(2, 'Pharma Inc', 'reefer RTR456 broke 34degF chicago pharma DESTROYED \$85k damage!', '2026-01-08 15:15:00'),
(3, 'AutoPlant LLC', 'palletjack WH1 broke!! SKU#78901 steel 48PLTS \$15K value WAREHOUSE1', '2026-01-08 16:00:00'),
(4, 'Hospital Sys', 'RES DLV fail 123MainSt#4B LG REQ \$2200 residential LIFTGATE NEEDED asap!!', '2026-01-08 16:45:00'),
(5, 'DataCenterOps', 'EXPD PO-98765 INV#123 ovrchg \$2450 68degF expedite FAIL billing error', '2026-01-08 17:20:00'),
(6, 'OilRigSupply', 'HAZMAT CL8 UN#1830 40F CORROSIVE ups carrier \$22K tempcontrol FAIL', '2026-01-08 17:55:00'),
(7, 'RetailHQ', 'LG CarrierB NE resdlv PO22222 \$1100 3PL dispute liftgate service ok', '2026-01-08 18:30:00'),
(8, 'AirlineParts', 'EXP AIR ORD INV-888 FAA145 delayed \$7500 repair station expressfreight', '2026-01-08 19:05:00'),
(9, 'OfficeSupplyCo', 'GRND SKU45678 paper DC1 \$350 std delivery perfect', '2026-01-08 09:00:00'),
(10, 'MarketingDept', 'brochure PLT WH2 LTL PO-33333 \$1800 nohazmat std', '2026-01-08 10:30:00'),
(11, 'LegalTeam', 'docs PO#11111 INV777 \$950 sigpage missing mail std', '2026-01-08 11:15:00'),
(12, 'FinanceDept', 'INV999 corr \$1250.75 freight adj 72F temp ok', '2026-01-08 12:00:00');

-- ai_extract() extracts from TOTAL CHAOS (no regex could handle this)
SELECT 
  ticket_id, customer_name, description,
  
  -- Single call extracts ALL entities at once (most efficient)
  ai_extract(description, 
    ARRAY(
      'container_number', 'trailer_number', 'po_number', 'invoice_number', 
      'sku_number', 'dollar_value', 'temperature_f', 'hazmat_class', 'un_number'
    )
  ) AS extracted_entities

FROM lesteve_sandbox.bronze.ai_extract_tickets
ORDER BY ticket_id;


ticket_id,customer_name,description,extracted_entities
1,AcmeCorp,stuck CNTR-ABC-123 at LB port!!! need LG LTL $12k 12KUSD liftgate midwest,"List(CNTR-ABC-123, null, null, null, null, $12k, null, null, null)"
2,Pharma Inc,reefer RTR456 broke 34degF chicago pharma DESTROYED $85k damage!,"List(null, RTR456, null, null, null, $85k, 34degF, null, null)"
3,AutoPlant LLC,palletjack WH1 broke!! SKU#78901 steel 48PLTS $15K value WAREHOUSE1,"List(null, null, null, null, SKU#78901, $15K, null, null, null)"
4,Hospital Sys,RES DLV fail 123MainSt#4B LG REQ $2200 residential LIFTGATE NEEDED asap!!,"List(null, null, null, null, null, $2200, null, null, null)"
5,DataCenterOps,EXPD PO-98765 INV#123 ovrchg $2450 68degF expedite FAIL billing error,"List(null, null, PO-98765, INV#123, null, $2450, 68degF, null, null)"
6,OilRigSupply,HAZMAT CL8 UN#1830 40F CORROSIVE ups carrier $22K tempcontrol FAIL,"List(null, null, null, null, null, $22K, 40F, CL8, UN#1830)"
7,RetailHQ,LG CarrierB NE resdlv PO22222 $1100 3PL dispute liftgate service ok,"List(null, null, PO22222, null, null, $1100, null, null, null)"
8,AirlineParts,EXP AIR ORD INV-888 FAA145 delayed $7500 repair station expressfreight,"List(null, null, null, INV-888, null, $7500, null, null, null)"
9,OfficeSupplyCo,GRND SKU45678 paper DC1 $350 std delivery perfect,"List(null, null, null, null, SKU45678, $350, null, null, null)"
10,MarketingDept,brochure PLT WH2 LTL PO-33333 $1800 nohazmat std,"List(null, null, PO-33333, null, null, $1800, null, nohazmat, null)"


# ai_analyze_sentiment()

In [0]:
%sql
-- ai_analyze_sentiment() Demo: Customer Feedback
CREATE OR REPLACE TABLE lesteve_sandbox.bronze.customer_feedback (
  feedback_id INT, 
  customer_name STRING,
  feedback_text STRING,
  created_date TIMESTAMP
) USING DELTA;


-- 16 real 3PL customer feedback messages
INSERT INTO lesteve_sandbox.bronze.customer_feedback VALUES 
(1, 'Acme Corp', 'Liftgate service was perfect, driver very professional and on time!', '2026-01-08 08:15:00'),
(2, 'PharmaCorp', 'Reefer completely failed, lost \$50K in vaccines. Never again!', '2026-01-08 09:30:00'),
(3, 'AutoPlant', 'Pallet jack issue resolved quickly, great warehouse team', '2026-01-08 10:45:00'),
(4, 'Hospital', 'Residential delivery nightmare, liftgate broke at delivery', '2026-01-08 11:20:00'),
(5, 'DataCenter', 'Expedited shipping saved our cloud migration deadline!', '2026-01-08 12:10:00'),
(6, 'OilRig', 'HAZMAT handling was sloppy, paperwork completely wrong', '2026-01-08 13:25:00'),
(7, 'RetailHQ', 'Liftgate service CarrierB was excellent, very reliable', '2026-01-08 14:40:00'),
(8, 'Airline', 'Express freight delayed 3 days, missed FAA certification', '2026-01-08 15:55:00'),
(9, 'OfficeSupplies', 'Standard delivery exactly as promised, thank you', '2026-01-08 09:00:00'),
(10, 'Marketing', 'Brochures arrived damaged, poor pallet protection', '2026-01-08 10:30:00'),
(11, 'Legal', 'Contract docs delivered perfectly, right to legal team', '2026-01-08 11:15:00'),
(12, 'Finance', 'Invoice correction took 3 weeks, billing nightmare', '2026-01-08 12:00:00'),
(13, 'Warehouse1', 'Temperature control perfect, pharma shipment 34F exactly', '2026-01-08 16:20:00'),
(14, 'CustomerSvc', 'Residential delivery driver rude, left at wrong address', '2026-01-08 17:35:00'),
(15, 'SupplyChain', 'Liftgate service on time every time, 5 stars!', '2026-01-08 18:50:00'),
(16, 'Operations', 'Expedited fee charged but delivered standard ground speed', '2026-01-08 19:05:00');

-- FIXED: Pre-compute sentiment WITH deterministic ORDER BY
WITH sentiment_analysis AS (
  SELECT 
    feedback_id, customer_name, feedback_text,
    ai_analyze_sentiment(feedback_text) AS sentiment,
    CASE 
      WHEN ai_analyze_sentiment(feedback_text) = 'negative' THEN 'PRIORITIZE'
      WHEN ai_analyze_sentiment(feedback_text) = 'positive' THEN 'NPS FOLLOWUP' 
      ELSE 'MONITOR'
    END AS action_required,
    ROW_NUMBER() OVER (ORDER BY feedback_id) as rn  -- Deterministic row number
  FROM lesteve_sandbox.bronze.customer_feedback
)
SELECT 
  feedback_id, customer_name, feedback_text, sentiment, action_required
FROM sentiment_analysis
ORDER BY 
  CASE sentiment
    WHEN 'negative' THEN 1
    WHEN 'mixed' THEN 2
    WHEN 'neutral' THEN 3
    WHEN 'positive' THEN 4
  END, feedback_id;


feedback_id,customer_name,feedback_text,sentiment,action_required
2,PharmaCorp,"Reefer completely failed, lost $50K in vaccines. Never again!",negative,PRIORITIZE
4,Hospital,"Residential delivery nightmare, liftgate broke at delivery",negative,PRIORITIZE
6,OilRig,"HAZMAT handling was sloppy, paperwork completely wrong",negative,PRIORITIZE
8,Airline,"Express freight delayed 3 days, missed FAA certification",negative,PRIORITIZE
10,Marketing,"Brochures arrived damaged, poor pallet protection",negative,PRIORITIZE
12,Finance,"Invoice correction took 3 weeks, billing nightmare",negative,PRIORITIZE
14,CustomerSvc,"Residential delivery driver rude, left at wrong address",negative,PRIORITIZE
16,Operations,Expedited fee charged but delivered standard ground speed,negative,PRIORITIZE
1,Acme Corp,"Liftgate service was perfect, driver very professional and on time!",positive,NPS FOLLOWUP
3,AutoPlant,"Pallet jack issue resolved quickly, great warehouse team",positive,NPS FOLLOWUP


# ai_gen()

In [0]:
%sql
-- ai_gen() Demo: Auto-Generate  Responses & Documentation
CREATE OR REPLACE TABLE lesteve_sandbox.bronze.ai_gen_tickets (
  ticket_id INT, 
  customer_name STRING,
  description STRING,
  priority STRING
) USING DELTA;


-- 10 tickets needing auto-generated responses/escalations
INSERT INTO lesteve_sandbox.bronze.ai_gen_tickets VALUES 
(1, 'Acme Corp', 'Container ABC123 stuck at Long Beach 3 days, production down', 'P1-Critical'),
(2, 'PharmaCorp', 'Reefer failed 34F, \$50K vaccines destroyed FDA audit tomorrow', 'P1-Critical'),
(3, 'AutoPlant', 'Pallet jack broken Warehouse1, 48 steel beams stranded', 'P2-High'),
(4, 'Hospital', 'Residential delivery failed 123 Main St Apt 4B liftgate needed', 'P2-High'),
(5, 'DataCenter', 'Expedited PO#98765 overcharged INV123 \$2,450', 'P3-Medium'),
(6, 'OilRig', 'HAZMAT Class 8 UN1830 paperwork wrong, safety violation', 'P1-Critical'),
(7, 'RetailHQ', 'Liftgate CarrierB excellent service, want same driver future', 'P4-Low'),
(8, 'Airline', 'Express freight 3 days late, missed FAA Part 145 deadline', 'P1-Critical'),
(9, 'OfficeSupplies', 'Paper reams SKU45678 perfect condition on time', 'P4-Low'),
(10, 'Finance', 'Invoice INV999 correction needed, charged expedited fee wrongly', 'P3-Medium');

-- ai_gen(): Generate business responses from ticket context
SELECT 
  ticket_id, customer_name, description, priority,
  
  -- Customer response email
  ai_gen(CONCAT(
    'Write professional 3PL customer email response for ticket priority ', priority, '. ',
    'Customer: ', customer_name, '. ',
    'Tone: Apologetic if P1/P2, Grateful if P4. ',
    'Include resolution timeline and \$100 credit offer for P1/P2. ',
    'Ticket: ', description
  )) AS customer_email_response,
  
  -- Ops team escalation message  
  ai_gen(CONCAT(
    'Generate an internal Teams message for the 3PL ops team. ',
    'Do NOT start with any sample or introductory phrase. ',
    'Include ticket priority, key issue, and action needed. ',
    'Ticket: ', description
  )) AS ops_escalation

FROM lesteve_sandbox.bronze.ai_gen_tickets
ORDER BY 
  CASE priority 
    WHEN 'P1-Critical' THEN 1
    WHEN 'P2-High' THEN 2
    WHEN 'P3-Medium' THEN 3
    ELSE 4 
  END, ticket_id;

ticket_id,customer_name,description,priority,customer_email_response,ops_escalation
1,Acme Corp,"Container ABC123 stuck at Long Beach 3 days, production down",P1-Critical,"Subject: Urgent: Resolution for Container ABC123 at Long Beach - Acme Corp Dear Acme Corp Team, I am writing to acknowledge and apologize for the critical issue with Container ABC123, which has been stuck at the Long Beach terminal for the past three days, resulting in production downtime for your operations. I understand the severity of this situation and the impact it has on your business. Please accept our sincerest apologies for the inconvenience this has caused. Our team has been working diligently to resolve this matter as quickly as possible. After investigating the cause of the delay, we have identified the root issue and are taking immediate action to expedite the container's release. We anticipate that Container ABC123 will be released from the terminal within the next 24 hours, and we will ensure that it is prioritized for transportation to its final destination. To provide a more detailed timeline: * Within the next 12 hours, our team will complete the necessary paperwork and clearances to facilitate the container's release. * Once released, the container will be transported to its final destination, with an estimated delivery time of 48 hours from the time of release. As a gesture of goodwill and a commitment to making things right, we would like to offer Acme Corp a $100 credit towards your next invoice. This credit is a token of our apology for the disruption caused and our appreciation for your patience and understanding in this matter. If you have any questions or concerns, please do not hesitate to reach out to me directly. Your satisfaction is our top priority, and we are committed to preventing such issues in the future. Thank you for your understanding, and we look forward to resolving this matter promptly. Best regards, [Your Name] Customer Support Specialist [3PL Company Name]","Priority: High Issue: Container ABC123 has been stuck at the Long Beach port for 3 days, resulting in production downtime Action Needed: Ops team to immediately contact the shipping carrier and port authorities to investigate the cause of the delay and expedite the release of the container. Provide an update on the expected release date and develop a contingency plan to minimize further production disruptions."
2,PharmaCorp,"Reefer failed 34F, $50K vaccines destroyed FDA audit tomorrow",P1-Critical,"Subject: Urgent: Reefer Failure and Vaccine Destruction - Resolution and Support for PharmaCorp Dear PharmaCorp Team, I am writing to express my sincerest apologies for the critical issue that has occurred with the reefer failure, resulting in the destruction of $50,000 worth of vaccines. I understand the severity of this situation, especially with the FDA audit scheduled for tomorrow. Please know that we take full responsibility for this incident and are committed to making it right. Our team has immediately initiated an investigation into the cause of the reefer failure and is working diligently to prevent such incidents in the future. In the meantime, we are taking the following steps to support you: 1. **Immediate Replacement**: We are arranging for an emergency replacement of the destroyed vaccines, if possible, to ensure you have the necessary stock for the FDA audit. 2. **Expedited Shipping**: We will expedite the shipping of the replacement vaccines to ensure they arrive at your facility as soon as possible. 3. **Dedicated Support**: A dedicated support specialist will be assigned to your account to ensure seamless communication and resolution of this issue. We anticipate resolving this issue within the next 24 hours, with the following timeline: * Replacement vaccines to be shipped within the next 12 hours * Expected delivery of replacement vaccines within 12-18 hours * Continuous updates and communication from our dedicated support specialist As a gesture 